In [1]:
!pip install rouge
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging
!pip install wandb

In [31]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re

from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm

In [3]:
model_checkpoints = "gogamza/kobart-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints) 

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (..) 제거
    sentence = re.sub(r'[#@]+[가-힣A-Za-z#]+', ' ', sentence)
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    # 스폐셜 토큰 적용할 거면 여기 위에 영어 외 문자 공백으로 만들때 스폐셜 토큰을 넘어갈 수 있도록 지정해주면된다.
    # 그리고 세번째 정규표현식을 지워야 할 것이다. 
    return sentence

In [5]:
train_df = pd.read_csv('data/train_total.csv')
val_df = pd.read_csv('data/val_total.csv')

In [6]:
train_Category = train_df['Category'].unique()
val_Category = val_df['Category'].unique()
def categori_ext(data, Category, tv):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
    return df

In [7]:
train_df = categori_ext(train_df, train_Category, 'train')
val_df = categori_ext(val_df, val_Category, 'val')

In [8]:
train_df.head()

,Id,Text,Summary,Category
0,fd321028-d5b4-55f7-9e20-2eaa262f9154,"['그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?', '우리 비행...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.",상거래(쇼핑)
1,c51be2e4-c8d0-5cea-b1ae-cde1fe8f8ab6,"['Kf마스크만 5부제 하는거지?', '응. 면마스크는 아무때나 사도될껀?', '면...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.,상거래(쇼핑)
2,e90e721f-00d1-5114-aa5d-5f1061472a29,['아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애'...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...,상거래(쇼핑)
3,b215f3a2-d647-59f9-8410-1274ee5edd97,"['칫솔사야하는데 쓱으로 살까?', '뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ', '아 그...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.,상거래(쇼핑)
4,0bda61b6-1396-5a2a-a049-0b4035e40d59,"['잠도안오네ㅐ얼릉 고구마츄 먹고싶단', '그게 그렇게 맛있었어??? 아주 여보 빼...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.,상거래(쇼핑)


In [9]:
# 전체 Text 데이터에 대한 전처리 (1)
train_text = []
train_summary = []

for tt in tqdm(train_df['Text']):
    train_text.append(preprocess_sentence(tt))

for ts in tqdm(train_df['Summary']):
      train_summary.append(preprocess_sentence(ts))


100%|██████████| 13994/13994 [00:00<00:00, 72843.93it/s]


In [10]:
# 전체 Text 데이터에 대한 전처리 (1)
val_text = []
val_summary = []

for vt in tqdm(val_df['Text']):
    val_text.append(preprocess_sentence(vt))

for vs in tqdm(val_df['Summary']):
      val_summary.append(preprocess_sentence(vs))


100%|██████████| 1746/1746 [00:00<00:00, 67865.73it/s]


In [11]:
del train_text[0]
del train_summary[0]
del val_text[0]
del val_summary[0]

In [12]:
train_df = pd.DataFrame(zip(train_text,train_summary), columns=['Text', 'Summary'])
val_df = pd.DataFrame(zip(val_text,val_summary), columns=['Text', 'Summary'])

In [13]:
train_df.head()

,Text,Summary
0,kf마스크만 5부제 하는거지 응 면마스크는 아무때나 사도될껀 면마스크말고 부직포 마...,비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다
1,아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애 그러...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
2,칫솔사야하는데 쓱으로 살까 뭘 칫솔사는것까지 물어보시남 아 그 왕칫솔 또 사려나 싶...,칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계 가자고 했다
3,잠도안오네 얼릉 고구마츄 먹고싶단 그게 그렇게 맛있었어 아주 여보 빼이보릿 되버렸네...,잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다
4,나 오늘 빼빼로 25000원어치 삿는데 5000원 상품권받앗엌 개이득 머그렇게많이삿...,오늘 이마트에 가서 주임님 심부름으로 빼빼로 25000원어치를 샀는데 5000원 상...


In [14]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)
test_samples = Dataset.from_pandas(val_df)

In [15]:
print(train_data)
print(val_data)
print(test_samples)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 13993)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 1745)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 1745)


In [16]:
max_input = 128
max_target = 32
batch_size = 4
ignore_index = -100# tokenizer.pad_token_id

In [17]:
 def add_ignored_data(inputs, max_len, ignore_index):
        if len(inputs) < max_len:
            pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs

In [18]:
def add_padding_data(inputs, max_len):
        pad_index = tokenizer.pad_token_id
        if len(inputs) < max_len:
            pad = [pad_index] *(max_len - len(inputs))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs 

In [19]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []

    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i]), max_input))
    for i in range(len(data_to_process['Summary'])):
        label_id.append(tokenizer.encode(data_to_process['Summary'][i]))  
        label_id[i].append(tokenizer.eos_token_id)  
    for i in range(len(data_to_process['Summary'])):  
        dec_input_id = [tokenizer.eos_token_id]
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Summary'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

In [20]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [21]:
# # set special tokens
#from transformers import EncoderDecoderConfig
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 32 # 256은 쿠다 메모리 오류 생김
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 5
model.config.length_penalty = 2.0
model.config.num_beams = 5
#model.config.suppress_tokens = [23782, 14338, 22554, 234]


In [22]:
model.config

BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
 

In [23]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)
   

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoint/stopword_test",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=64,
    learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    logging_steps=500,
    save_total_limit=3,

    #evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    #load_best_model_at_end = True,

)

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?
"""
DataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . 
또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.
tokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.
"""

'\nDataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . \n또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.\ntokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.\n'

In [26]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
   # callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13993
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4375
  Number of trainable parameters = 123859968
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,4.753400
1000,3.710000
1500,3.443900
2000,3.276000
2500,3.116100
3000,2.957000
3500,2.914100
4000,2.781800


Saving model checkpoint to checkpoint/stopword_test/checkpoint-500
Configuration saved in checkpoint/stopword_test/checkpoint-500/config.json
Model weights saved in checkpoint/stopword_test/checkpoint-500/pytorch_model.bin
tokenizer config file saved in checkpoint/stopword_test/checkpoint-500/tokenizer_config.json
Special tokens file saved in checkpoint/stopword_test/checkpoint-500/special_tokens_map.json
Saving model checkpoint to checkpoint/stopword_test/checkpoint-1000
Configuration saved in checkpoint/stopword_test/checkpoint-1000/config.json
Model weights saved in checkpoint/stopword_test/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in checkpoint/stopword_test/checkpoint-1000/tokenizer_config.json
Special tokens file saved in checkpoint/stopword_test/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to checkpoint/stopword_test/checkpoint-1500
Configuration saved in checkpoint/stopword_test/checkpoint-1500/config.json
Model weights saved in checkpoint

TrainOutput(global_step=4375, training_loss=3.3184619140625, metrics={'train_runtime': 2462.0701, 'train_samples_per_second': 28.417, 'train_steps_per_second': 1.777, 'total_flos': 5332526412595200.0, 'train_loss': 3.3184619140625, 'epoch': 5.0})

In [28]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1745
  Batch size = 64


Trainer is attempting to log a value of "{'r': 0.2317488325440182, 'p': 0.23724880799787368, 'f': 0.2276783764590438}" of type <class 'dict'> for key "eval/rouge-1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.08670285190083027, 'p': 0.08956570706353374, 'f': 0.08503088446710783}" of type <class 'dict'> for key "eval/rouge-2" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'r': 0.22017302207895775, 'p': 0.2251341575506365, 'f': 0.21615822355821032}" of type <class 'dict'> for key "eval/rouge-l" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 3.4915246963500977,
 'eval_rouge-1': {'r': 0.2317488325440182,
  'p': 0.23724880799787368,
  'f': 0.2276783764590438},
 'eval_rouge-2': {'r': 0.08670285190083027,
  'p': 0.08956570706353374,
  'f': 0.08503088446710783},
 'eval_rouge-l': {'r': 0.22017302207895775,
  'p': 0.2251341575506365,
  'f': 0.21615822355821032},
 'eval_runtime': 156.0038,
 'eval_samples_per_second': 11.186,
 'eval_steps_per_second': 0.179,
 'epoch': 5.0}

In [29]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, num_beams=5,
                            attention_mask=attention_mask,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            eos_token_id=tokenizer.eos_token_id,)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)


loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/f9f2ec35d3c32a1ecc7a3281f9626b7ec1913fed/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "

In [30]:
#summaries_before_tuning = []
#for test_sample in tqdm(test_samples):
#    summaries_before_tuning.append(generate_summary(test_sample, model_before_tuning)[1])
#summaries_before_tuning = list(itertools.chain(*summaries_before_tuning))    
    
summaries_after_tuning=[]
for test_sample in tqdm(test_samples):
    summaries_after_tuning.append(generate_summary(test_sample, model)[1])


  0%|          | 0/1745 [00:00<?, ?it/s]/opt/conda/lib/python3.9/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 32 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1745/1745 [06:05<00:00,  4.77it/s]


NameError: name 'itertools' is not defined

In [32]:
summaries_after_tuning = list(itertools.chain(*summaries_after_tuning))

In [33]:
rouge.get_scores(summaries_after_tuning, test_samples["Summary"], avg=True)

{'rouge-1': {'r': 0.20882171155596074,
  'p': 0.23552698144599316,
  'f': 0.21425186937436974},
 'rouge-2': {'r': 0.07532400764069774,
  'p': 0.08611587828805094,
  'f': 0.07730825523470342},
 'rouge-l': {'r': 0.19856471372992923,
  'p': 0.2239313106505082,
  'f': 0.20373713977528604}}

In [35]:
for i in range(0, len(summaries_after_tuning), 100):
    print('idx_{} '.format(i))
   # print("Summary before \n", summaries_before_tuning[i])
    print()
    print("Summary after \n", summaries_after_tuning[i])
    print()
    print("Target summary \n", test_samples["Summary"][i])
    print()
    print('Text', test_samples["Text"][i])
    print('-'*100)
    print()  

idx_0 

Summary after 
 이력서 쓰고 영어도 해야 하고 회사 선택도 잘 해야 한다

Target summary 
 이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다

Text 너는 잘가라 회사 선택 잘해 알겠어 많이 힘들구나 나도 이제 이력서쓰고 영어도 해야해 우리 똥갱애지 화이팅 첫 회사 잘들어가야한다 좋은 시절 다 지났다
----------------------------------------------------------------------------------------------------

idx_100 

Summary after 
 이력서를 들고 간다고 하니 그냥 가겠다고 한다

Target summary 
 수업을 들으며 돈을 모으기 위해 아르바이트를 할 수 있는지 여부에 대해 이야기한다

Text 너이력서들고감 아니 그냥갔어 나도 그냥 가야겟구만 이따 저나가능 언제 지금 지금은 됨 야그럼 너 차피 학원다니기 시작하면 알바 하나도 못하는겨 아니지 학원다니는게아니라 수업듣는거야 근데 지금개강하는건 수학1 이라서 이건 들을구잇는데 2월에개강하는게 미적분 인데 이건 그냥 포기할려고 일단 돈 모으는게 맞는거같음
----------------------------------------------------------------------------------------------------

idx_200 

Summary after 
 구미에 갔을 때 마스크를 쓰고 싸구려 장갑 하나 사서 끼고 다녀 마스크를 쓰고 갔다고 한다

Target summary 
 방 문제로 구미에 가면서 케이에프 마스크를 썼고 내일 구리로 다시 갈 거라고 한다

Text 왜 구미 갔어 때가 어느 때인데 간거야 마스크 쓰고 싸구려 장갑 하나 사서 끼고 다녀 마스크 쓰고 갔고 kf94도 2개 챙겨갔으 방때메 가야한대 낼 올라온대 kf94가 뭐야 아 한국인증필터마스크 지수 94는 초미세먼지까지 막아주는 마스크 되도록 대구는 머물지 말